In [ ]:
## 필요한 라이브러리 로딩
import pandas as pd
from datetime import datetime, timedelta
from pandas.api.types import CategoricalDtype
pd.set_option('mode.chained_assignment',  None)

## 1. covid19 원본 데이터셋 로딩
## covid19 데이터 로딩(파일을 다운로드 받은 경우)
df_covid19 = pd.read_csv("D:/R/data/Rnpy//owid-covid-data.csv")

## covid19 데이터 로딩(온라인에서 바로 로딩할 경우)
##df_covid19 = pd.read_csv(“https://covid.ourworldindata.org/data/owid-covid-data.csv”)
## 2. 전체 데이터셋 중 최근 100일간의 데이터를 필터링한 df_covid19_100 생성
##df_covid19['date']를 datetime으로 변환
df_covid19['date'] = pd.to_datetime(df_covid19['date'], format="%Y-%m-%d")

## 대륙 데이터와 최종 데이터로부터 100일 전 데이터 필터링
df_covid19_100 = df_covid19[(df_covid19['iso_code'].isin(['KOR', 'OWID_ASI', 'OWID_EUR', 
'OWID_OCE', 'OWID_NAM', 'OWID_SAM', 'OWID_AFR'])) 
& (df_covid19['date'] >= (max(df_covid19['date']) - timedelta(days = 100)))]

## 대륙명을 한글로 변환
df_covid19_100.loc[df_covid19_100['location'] == 'South Korea', "location"] = '한국'
df_covid19_100.loc[df_covid19_100['location'] == 'Asia', "location"] = '아시아'
df_covid19_100.loc[df_covid19_100['location'] == 'Europe', "location"] = '유럽'
df_covid19_100.loc[df_covid19_100['location'] == 'Oceania', "location"] = '오세아니아'
df_covid19_100.loc[df_covid19_100['location'] == 'North America', "location"] = '북미'
df_covid19_100.loc[df_covid19_100['location'] == 'South America', "location"] = '남미'
df_covid19_100.loc[df_covid19_100['location'] == 'Africa', "location"] = '아프리카'

## 이산형 변수 설정
ord = CategoricalDtype(categories = ['한국', '아시아', '유럽', '북미', '남미', 
'아프리카','오세아니아'], ordered = True)

df_covid19_100['location'] = df_covid19_100['location'].astype(ord)

## date로 정렬
df_covid19_100 = df_covid19_100.sort_values(by = 'date')

## 3. df_covid19_100을 한국과 각 대륙별 열로 배치한 넓은 형태의 데이터프레임으로 변환
df_covid19_100_wide = df_covid19_100.loc[:,['date', 'location', 'new_cases', 
'people_fully_vaccinated_per_hundred']].rename(columns={'new_cases':'확진자', 
'people_fully_vaccinated_per_hundred':'백신접종완료자'})

df_covid19_100_wide = df_covid19_100_wide.pivot(index='date', columns='location', 
values=['확진자', '백신접종완료자']).sort_values(by = 'date')

df_covid19_100_wide.columns = ['확진자_한국', '확진자_아시아', '확진자_유럽', '확진자_북미',  
'확진자_남미', '확진자_아프리카', '확진자_오세아니아', '백신접종완료자_한국', 
'백신접종완료자_아시아', '백신접종완료자_유럽', '백신접종완료자_북미', 
'백신접종완료자_남미', '백신접종완료자_아프리카', '백신접종완료자_오세아니아']

## 4. covid19 데이터를 국가별로 요약한 df_covid19_stat 생성
df_covid19_stat = df_covid19.groupby(['iso_code', 'continent', 'location'], dropna=True).agg(
    인구수 = ('population', 'max'), 
    전체사망자수 = ('new_deaths', 'sum'), 
    백신접종자완료자수 = ('people_fully_vaccinated', 'max'), 
    인구백명당백신접종완료율 = ('people_fully_vaccinated_per_hundred', 'max'), 
    인구백명당부스터접종자수 = ('total_boosters_per_hundred', 'max')).reset_index()

df_covid19_stat['십만명당사망자수'] = round(df_covid19_stat['전체사망자수'] / 
                                    df_covid19_stat['인구수'] * 100000, 5)

df_covid19_stat['백신접종완료율'] = df_covid19_stat['백신접종자완료자수'] / \
df_covid19_stat['인구수']

## 여백 설정을 위한 변수 설정
margins_P = {'t' : 50, 'b' : 25, 'l' : 25, 'r' : 25}

In [ ]:
## 파이썬 코드
## 대학 학과 취업률 데이터 로딩
df_취업률 = pd.read_excel("D:/R/data/Rnpy/2021년 학과별 고등교육기관 취업통계.xlsx", \
                       ## '학과별' 시트의 데이터를 불러오는데,
                       sheet_name = '학과별', \
                       ## 앞의 13행을 제외하고 \
                       skiprows=(13), \
                       ## 첫 번째 행은 열 이름으로 설정
                       header = 0)
## df_취업률에서 첫 번째부터 9번째까지의 열과 '계'로 끝나는 열을 선택하여 다시 df_취업률에 저장
df_취업률 = pd.concat([df_취업률.iloc[:, 0:8],
df_취업률.loc[:, df_취업률.columns.str.endswith('계')], \
                    df_취업률.loc[:, '입대자']], \
                   axis = 1)
## df_취업률에서 졸업자가 500명 이하인 학과 중 25% 샘플링
df_취업률_500 = df_취업률.loc[(df_취업률['졸업자_계'] < 500)]
df_취업률_500 = df_취업률_500.iloc[range(0, len(df_취업률_500.index) , 4)]
df_취업률_500 = df_취업률_500.rename(columns = {'졸업자_계':'졸업자수', '취업률_계': 
'취업률', '취업자_합계_계':'취업자수'})

In [ ]:
## 파이썬에서 Plotly 라이브러리 모듈 로딩
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
###############################################################################################
## chap 11. 시각화컨트롤
###############################################################################################

In [ ]:
import pandas as pd
import plotly.graph_objects as go
df_충원율_control = pd.read_excel(
    "D:/R/data/Rnpy/고등 주요 01-시도별 신입생 충원율(2010-2022)_220825y.xlsx",
    sheet_name = 'Sheet1', skiprows=(6), header = 0)

df_충원율_control = df_충원율_control.iloc[:, [0, 1, 4]]

df_충원율_control.columns = ('연도', '지역', '신입생충원율')

df_충원율_control = df_충원율_control.pivot(
    index = '지역', columns = '연도', values = '신입생충원율').reset_index()

In [ ]:
fig = go.Figure()

## 데이터가 표시되는 bar 트레이스 생성
fig.add_trace(go.Bar(
    x = df_충원율_control['지역'], y = df_충원율_control[2022],
    text = df_충원율_control[2022],
    texttemplate = '%{text:.1f}%', textposition = 'outside'
    ))

## 버튼 제목이 표시되는 주석 레이어 생성
fig.add_annotation(x = -0.1, y = 0.85, text = '<b>연도</b>',
    xanchor = 'center', yanchor = 'middle',
    yref='paper', xref='paper', showarrow=False)

## 버튼 생성
fig.update_layout(
    title = dict(text = "2022년 지역별 충원율", x = 0.5),
    xaxis = dict(domain = (0.1, 1), categoryorder = "total descending"),
    yaxis = dict(title = "충원율(%)"),
    updatemenus =[ ## updatemenus의 할당을 위한 리스트 정의
        dict( ## updatemenus에 할당된 버튼 그룹 딕서너리 설정
            type = "buttons", y = 0.8,
            buttons = [ ## 버튼 그룹에 생성되는 다섯 개의 버튼 설정
                dict(method = "restyle",
                     ## 첫 번째 버튼의 args 설정
                     args = [dict(y = [df_충원율_control[2018]], 
                                  text = [df_충원율_control[2018]])],
                     label = "2018년"),
                dict(method = "restyle",
                     ## 두 번째 버튼의 args 설정
                     args = [dict(y = [df_충원율_control[2019]], 
                                  text = [df_충원율_control[2019]])],
                     label = "2019년"),
                dict(method = "restyle",
                     ## 세 번째 버튼의 args 설정
                     args = [dict(y = [df_충원율_control[2020]], 
                                  text = [df_충원율_control[2020]])],
                     label = "2020년"),
                dict(method = "restyle",
                     ## 네 번째 버튼의 args 설정
                     args = [dict(y = [df_충원율_control[2021]], 
                                  text = [df_충원율_control[2021]])],
                     label = "2021년"),
                dict(method = "restyle",
                     ## 다섯 번째 버튼의 args 설정
                     args = [dict(y = [df_충원율_control[2022]], 
                                  text = [df_충원율_control[2022]])],
                     label = "2022년")
                ])
        ])

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x = df_충원율_control['지역'], y = df_충원율_control[2022],
    text = df_충원율_control[2022], texttemplate = '%{text:.1f}%',
    textposition = 'outside'))

fig.add_annotation(x = -0.1, y = 0.85, text = '<b>연도</b>',
                   xanchor = 'center', yanchor = 'middle',
                   yref='paper', xref='paper', showarrow=False)

fig.update_layout(title = dict(text = "2022년 지역별 충원율", x = 0.5),
                  xaxis = dict(domain = (0.1, 1), 
                               categoryorder = "total descending"),
                  yaxis = dict(title = "충원율(%)"),
                  updatemenus =[ ## updatemenus의 할당을 위한 리스트 정의
                      dict( ## updatemenus에 할당된 버튼 그룹 딕서너리 설정
                          type = "buttons", y = 0.8,
                          buttons = [ ## 버튼 그룹에 생성되는 다섯 개의 버튼 설정
                              dict(method = "relayout",
                                   ## 첫 번째 버튼의 args 설정
                                   args = [
                                       {'title.text' : '2018년 지역별 충원율'}], 
                                   label = "2018년"),
                              dict(method = "relayout",
                                   ## 두 번째 버튼의 args 설정
                                   args = [
                                       {'title.text' : '2019년 지역별 충원율'}], 
                                   label = "2019년"),
                              dict(method = "relayout",
                                   ## 세 번째 버튼의 args 설정
                                   args = [
                                       {'title.text' : '2020년 지역별 충원율'}], 
                                   label = "2020년"),
                              dict(method = "relayout",
                                   ## 네 번째 버튼의 args 설정
                                   args = [
                                       {'title.text' : '2021년 지역별 충원율'}], 
                                   label = "2021년"),
                              dict(method = "relayout",
                                   ## 다섯 번째 버튼의 args 설정
                                   args = [
                                       {'title.text' : '2022년 지역별 충원율'}], 
                                   label = "2022년")
                              ])
                  ])

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x = df_충원율_control['지역'], y = df_충원율_control[2022],
    text = df_충원율_control[2022], texttemplate = '%{text:.1f}%',
    textposition = 'outside'
    ))

fig.add_annotation(x = -0.1, y = 0.85, text = '<b>연도</b>',
                   xanchor = 'center', yanchor = 'middle',
                   yref='paper', xref='paper', showarrow=False)

fig.update_layout(title = dict(text = "2022년 지역별 충원율", x = 0.5),
                  xaxis = dict(domain = (0.1, 1), 
                               categoryorder = "total descending"),
                  yaxis = dict(title = "충원율(%)"),
                  updatemenus =[ ## updatemenus의 할당을 위한 리스트 정의
                      dict( 
                          ## updatemenus에 할당된 드롭다운 딕서너리 설정(드롭다운이 default)
                          y = 0.8,
                          buttons = [ ## 드롭다운 컨트롤에 생성되는 다섯 개의 메뉴 설정
                              dict(method = "update",
                                   args = [dict(y = [df_충원율_control[2018]],
                                                ## 첫 번째 메뉴의 data 속성에 대한 args 설정
                                                text = [df_충원율_control[2018]]),
                                           {'title.text' : '2018년 지역별 충원율'}],
                                   ## 첫 번째 메뉴의 layout 속성에 대한 args 설정
                                   label = "2018년"),
                              dict(method = "update",
                                   args = [dict(y = [df_충원율_control[2019]],
                                                ## 두 번째 메뉴의 data 속성에 대한 args 설정
                                                text = [df_충원율_control[2019]]),
                                           {'title.text' : '2019년 지역별 충원율'}],
                                   ## 두 번째 메뉴의 layout 속성에 대한 args 설정
                                   label = "2019년"),
                              dict(method = "update",
                                   args = [dict(y = [df_충원율_control[2020]],
                                                ## 세 번째 메뉴의 data 속성에 대한 args 설정
                                                text = [df_충원율_control[2020]]),
                                           {'title.text' : '2020년 지역별 충원율'}],
                                   ## 세 번째 메뉴의 layout 속성에 대한 args 설정
                                   label = "2020년"),
                              dict(method = "update",
                                   args = [dict(y = [df_충원율_control[2021]],
                                                ## 네 번째 메뉴의 data 속성에 대한 args 설정
                                                text = [df_충원율_control[2021]]),
                                           {'title.text' : '2021년 지역별 충원율'}],
                                   ## 네 번째 메뉴의 layout 속성에 대한 args 설정
                                   label = "2021년"),
                              dict(method = "update",
                                   args = [dict(y = [df_충원율_control[2022]],
                                                ## 다섯 번째 메뉴의 data 속성에 대한 args 설정
                                                text = [df_충원율_control[2022]]),
                                           {'title.text' : '2022년 지역별 충원율'}],
                                   ## 다섯 번째 메뉴의 layout 속성에 대한 args 설정
                                   label = "2022년")
                              ]) ])

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x = df_충원율_control['지역'], y = df_충원율_control[2022],
    text = df_충원율_control[2022], texttemplate = '%{text:.1f}%',
    textposition = 'outside'
    ))

steps = [
    dict(args = [dict(y = [df_충원율_control[2018]],
                      text = [df_충원율_control[2018]]),
                 {'title.text' : '2018년 지역별 충원율'}],
         label = "2018", method = "update", value = "1"
    ),
    dict(args = [dict(y = [df_충원율_control[2019]],
                      text = [df_충원율_control[2019]]),
                 {'title.text' : '2019년 지역별 충원율'}],
         label = "2019", method = "update", value = "2"
    ),
    dict(args = [dict(y = [df_충원율_control[2020]],
                      text = [df_충원율_control[2020]]),
                 {'title.text' : '2020년 지역별 충원율'}],
         label = "2020", method = "update", value = "3"
    ),
    dict(args = [dict(y = [df_충원율_control[2021]],
                      text = [df_충원율_control[2021]]),
                 {'title.text' : '2021년 지역별 충원율'}],
         label = "2021", method = "update", value = "4"
    ),
    dict(args = [dict(y = [df_충원율_control[2022]],
                      text = [df_충원율_control[2022]]),
                 {'title.text' : '2022년 지역별 충원율'}],
         label = "2022", method = "update", value = "5"
    )]

fig.update_layout(title = dict(text = "연도별 충원율", x = 0.5),
    xaxis = dict(categoryorder = "total descending"),
    yaxis = dict(title = "충원율(%)"),
    sliders = [dict(
    dict(active = 5,
         currentvalue = dict(prefix = "연도: "),
         pad = dict(t = 60),
         steps = steps))])

fig.show()

In [ ]:
###############################################################################################
## chap 14. Plotly 배포
###############################################################################################

In [ ]:
fig = go.Figure()

fig.add_trace(
    {'type' : 'scatter', 'mode' : 'markers',
    'x': df_취업률_500['졸업자수'], ## X, Y 축에 매핑되는 변수 설정
    'y': df_취업률_500['취업자수']})

fig.write_image("fig.png")

In [ ]:
fig.write_html("fig.html")

In [ ]:
import chart_studio
chart_studio.tools.set_credentials_file(username='차트 스튜디오의 사용자 이름', api_key='차트 스튜디오의 API Key값')

In [ ]:
{
"username": "차트 스튜디오의 사용자 이름",
"api_key": "차트 스튜디오의 API Key값"
}

In [ ]:
import chart_studio.plotly as py
py.plot(fig, filename = '업로드할 파일 이름', auto_open=True)